In [1]:
# First approach to Smart Charging Phase 1 using PuLP.
# Modeled as a PuLP optimisation blending problem
# Started 20 Aug 2020
# Author: Sofia Taylor, Flexible Power Systems
# test new laptop

import numpy as np
import pandas as pd
import datetime as dt
from pulp import *
import pickle
import global_variables as gv
import functions as f
import output_functions as of
import matplotlib as mpl
import matplotlib.pyplot as plt
#import glob
#import timeit
#import json 
import time
import random
import matplotlib.dates as mdates
import matplotlib.font_manager as font_manager
import lin_prog_functions as lpf


In [4]:
# Variables for grid search
run = 49
charger_power = [22, 45, 11, 7] # kW
caps = [200,250] #50 150 100 
grid_file_path = 'Outputs/Logs/grid_variables{}.csv'.format(run)
journeys = pickle.load(open('Outputs/journeys_range','rb'))
empty_profile = pickle.load(open('Outputs/empty_profile','rb'))
run +=16
for charger in charger_power:
    for capacity in caps:
        script_strt = time.process_time()
        print('Run:',run,'/ Charger:',charger,'/ Capacity:',capacity)
        site_capacity = {
            'opt': capacity,  # kWh (in a half-hour period so eq. 100 kW)
            'BAU': 10000,
            'BAU2': capacity
        }
        notes = 'Grid test' + str(charger) + ' / ' + str(capacity) +'\n'
        os.makedirs('Outputs/Logs/run{}'.format(run))
        grid_file = open(grid_file_path,'a')
        grid_file.write('\n' + str(run)+'\n'+str(charger) + '\n' + str(capacity) +'\n')
        profile_out, dates, bad_days, lpprob = lpf.optimise_range(
            journeys, 
            empty_profile, 
            charger, 
            site_capacity)

        range_profile, range_journeys, veh_profile, site_profile, days_summary, global_summary = of.summary_outputs(
        profile_out, 
            journeys, 
            dates)

        # print(global_summary)

        ################ OUTPUTS ####################
        # Make and save daily figures
        os.makedirs('Outputs/Logs/run{}/daily'.format(run))
        for date in dates:
            day = dt.datetime.combine(date, dt.datetime.min.time())
            day_profile = of.create_daily_summary(site_profile, day)
            fig_summary = of.summary_plot(day_profile)
            fig_summary.savefig(
                'Outputs/Logs/run{}/daily/fig{}.svg'.format(run,date))
            plt.close(fig_summary)
        # print('All good to 1')

        range_fig = of.daily_summary_plot(days_summary.fillna(0))
        range_fig.savefig(
            'Outputs/Logs/run{}/fig_range{}.svg'.format(run,run),
            bbox_inches = "tight")
        plt.close(range_fig)
        # print('All good with figs')

        
        # Create a list of settings
        with open('global_variables.py','r') as f:
            global_variables = f.read()

        with open('Outputs/Logs/run{}/variables{}.csv'.format(run,run),'a') as fi:
            fi.write(global_summary.to_string())
            fi.write('\nglobal_variabes.py:\n')
            fi.write(global_variables)
            fi.write(notes)
            fi.write(bad_days)

        # Write problem to an .lp file
        lpprob['opt'].writeLP("Outputs/Logs/run{}/multi_vehicle.lp".format(run))

        # Save dataframes
        pickle.dump(range_profile,open('Outputs/Logs/run{}/route_profiles{}'.format(run,run),'wb'))
        pickle.dump(veh_profile,open('Outputs/Logs/run{}/veh_profiles{}'.format(run,run),'wb'))
        pickle.dump(range_journeys,open('Outputs/Logs/run{}/journeys{}'.format(run,run),'wb'))
        pickle.dump(site_profile,open('Outputs/Logs/run{}/site_summary{}'.format(run,run),'wb'))
        pickle.dump(days_summary,open('Outputs/Logs/run{}/days_summary'.format(run),'wb'))
        grid_file.write(global_summary.to_string())
        grid_file.write('\n'+bad_days)
        runtime = time.process_time() - script_strt
        grid_file.write('\n'+str(runtime))
        print('Range:', gv.TIME_RANGE, 'Time:',time.process_time(),'Runtime:',runtime)
        run += 1
        grid_file.close()


1
2019-06-06 Status: 3 : 1 1 1
2019-06-07 Status: 3 : 1 1 1
2019-06-08 Status: 3 : 1 1 1
2019-06-09 Status: 3 : 1 1 1
2019-06-10 Status: 3 : 1 1 1
2019-06-11 Status: 3 : 1 1 1
2019-06-12 Status: 3 : 1 1 1
2019-06-13 Status: 3 : 1 1 1
2019-06-14 Status: 3 : 1 1 1
2019-06-15 Status: 3 : 1 1 1
2019-06-16 Status: 3 : 1 1 1
2019-06-17 Status: 3 : 1 1 1
2019-06-18 Status: 3 : 1 1 1
2019-06-19 Status: 3 : 1 1 1
2019-06-20 Status: 3 : 1 1 1
2019-06-21 Status: 3 : 1 1 1
2019-06-22 Status: 3 : 1 1 1
2019-06-23 Status: 3 : 1 1 1
2019-06-24 Status: 3 : 1 1 1
2019-06-25 Status: 3 : 1 1 1
2019-06-26 Status: 3 : 1 1 1
2019-06-27 Status: 3 : 1 1 1
2019-06-28 Status: 3 : 1 1 1
2019-06-29 Status: 3 : 1 1 1
2019-06-30 Status: 3 : 1 1 1
2019-07-01 Status: 3 : 1 1 1
2019-07-02 Status: 3 : 1 1 1
2019-07-03 Status: 3 : 1 1 1
2019-07-04 Status: 3 : 1 1 1
2019-07-05 Status: 3 : 1 1 1
2019-07-06 Status: 3 : 1 1 1
2019-07-07 Status: 3 : 1 1 1
2019-07-08 Status: 3 : 1 1 1
2019-07-09 Status: 3 : 1 1 1
2019-07-10 S

KeyboardInterrupt: 